In [ ]:
import json
import sys
import os
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize.stanford import StanfordTokenizer
from nltk.parse.corenlp import CoreNLPParser
from nltk.corpus import stopwords
from collections import defaultdict
from gensim import corpora
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec
import gensim
import re
import string
import nltk
import numpy as np
import pandas as pd
#nltk.download() #use if nltk cries for any language-specific docs

In [ ]:
#defaults for my own usecase, replace all accordingly
red_model_location = 'Models\word2vec\English_CoNLL17\ '#location of reference model
red_model_name = 'model.bin' #location of reference model binary
subreddits_json_location = 'Subs' #location of subreddit jsons

In [ ]:
"""ay=str()
for file in os.listdir('Models\word2vec\English CoNLL17 corpus - gensim skipgram - 10'):
    full_filename = "%s/%s" % ('Models\word2vec\English CoNLL17 corpus - gensim skipgram - 10', file)
    if full_filename.endswith('.txt'):
        with open(full_filename,'r', encoding="ISO-8859-1") as fi:
            ay=fi.readlines()
"""
subs=[]
for filename in os.listdir(subreddits_json_location):
    if filename.endswith(".json"):
        with open(os.path.join(subreddits_json_location, filename)) as json_file:
            json_text = json.load(json_file)
            print(filename)
            subs.append(json_text)

In [ ]:
#not used
def sub2vocab(sub):
    documents=sub2docs(sub)
    temp=[]
    stop = set(stopwords.words('english'))
    #for doc in documents:
        #print(word_tokenize(doc))
    texts =[
        [word for word in document.lower().split() if word not in stop]
        for document in documents
    ]
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1
    texts = [
        [token for token in text if frequency[token] > 1]
        for text in texts]
    return texts

In [ ]:
#not used even more
"""def sub2vocab(sub):
    raw=str()
    for i, j in sub.items():
        raw+=i
        raw+=' '
        for o,p in j.items():
            raw+=p
            raw+=' '
        raw+='\n'
    temp=[]
    stop = set(stopwords.words('english'))
    for i in word_tokenize(raw.lower()):
        if i not in stop:
            temp.append(i)
    frequency = defaultdict(int)
    for word in temp:
        frequency[word] += 1
    temp = set([token for token in temp if frequency[token] > 1])
    return temp"""

In [ ]:
def sub2docs(sub):
    documents=[]
    trans_table = str.maketrans("", "", string.punctuation)
    for i, j in sub.items():
        temp=re.sub(r'\d+','',i.lower().translate(trans_table)).replace('\n', '')
        documents.append(temp)
        for o,p in j.items():
            temp=re.sub(r'\d+', '',p.lower().translate(trans_table)).replace('\n', '')
            documents.append(temp)
    frequency = defaultdict(int)
    for document in documents:
        for token in document.split():
            frequency[token] += 1
    documents = [
        [token for token in document.split() if frequency[token] > 1]
        for document in documents]
    return documents

In [ ]:
def removePunctuation(words):
    trans_table = str.maketrans("", "", string.punctuation)
    return set([s.translate(trans_table) for s in words])

In [ ]:
def target_wv2distances(wv):
    dists=np.zeros((len(wv.vocab),len(wv.vocab)))
    words=[]
    for word in wv.vocab.keys():
        words.append(word)
    start=0
    for i in range(len(keys)):
        print(i,len(keys))
        for j in range(len(keys)):
            j+=start
            if j<len(keys):
                dists[i,j]=wv.n_similarity([words[i]],[words[j]])
        start+=1
    return dists

In [ ]:
def ref_wv2distances(target_wv,reference_wv):
    dists=np.zeros((len(target_wv.vocab),len(target_wv.vocab)))
    words=[]
    for word in target_wv.vocab.keys():
        words.append(word)
    start=0
    for i in range(len(keys)):
        print(i,len(keys))
        for j in range(len(keys)):
            j+=start
            try:
                if j<len(keys):
                    dists[i,j]=word_vectors.n_similarity([words[i]],[words[j]])
            except Exception as e:
                pass
                #print("type error: " + str(e))
        start+=1
    return dists


In [ ]:
def docs2wv(docs,name='model'):
    model = Word2Vec(docs, size=300, window=5, workers=3, sg=1, min_count=1, negative=8)
    if name!=None:
        model.save(name+".model")
    word_vectors = model.wv
    return word_vectors

In [ ]:
def remove_diagonal(dists):
    for i in range(dists.shape[0]):
        for j in range(dists.shape[1]):
            if i==j:
                dists[i,j]=0
    return dists

In [ ]:
target_wv=docs2wv(sub2docs(subs[0]),name='target')

In [ ]:
ref_wv=gensim.models.KeyedVectors.load_word2vec_format(os.path.join(os.path.dirname(red_model_name), red_model_location), binary=True).wv

In [ ]:
dists_ref=ref_wv2distances(target_wv,ref_wv)
dists_t=target_wv2distances(target_wv)

In [ ]:
dists_ref = remove_diagonal(dists_ref)

In [ ]:
dists_t  = remove_diagonal(dists_t)

In [ ]:
ranks=pd.DataFrame(columns=['Value'])
i=0
dists_comp=dists_t-dists_ref
for word in target_wv.vocab.keys():
    temp=np.sum(dists_comp[i,:])+np.sum(dists_comp[:,i])
    ranks.loc[word]=temp
    i+=1

In [ ]:
ranks.sort_values(['Value'])